<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [ ]:
##### Begin Working Here #####
#import pandas and dataset
import pandas as pd



In [ ]:
#Download the data
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

In [ ]:
#Exctract the downloaded data
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
replace LoanStats_2018Q4.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: LoanStats_2018Q4.csv    


In [ ]:
#checking the raw data file

#looking at the first five rows:
print("First five rows of data set:")
!head LoanStats_2018Q4.csv

#Looking at the last five rows:
print("Last five rows of data set:")
!tail LoanStats_2018Q4.csv

First five rows of data set:
Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","o

## Raw Data Analaysis
As we can see in the head and tail outputs of the data above, the first line is not part of the data table itself, and the last 2 lines are summations of the data included in the table, not part of the table itself.  We can remove these lines when importing the csv data using flags

In [ ]:
#Import the csv data, using the correct flags to remove the first line of information, and the last two lines of summation
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')

print(df.shape)
df.head()

(128412, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,...,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,16000,16000,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,...,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,8425,8425,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,NaN,NaN,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,...,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,...,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,...,14.3,1,0,138354,104590,36400,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Check the dataset for null values
print(df.shape)
df.isnull().sum().sort_values()

(128412, 144)


inq_fi                           0
mo_sin_old_rev_tl_op             0
delinq_amnt                      0
chargeoff_within_12_mths         0
acc_open_past_24mths             0
                             ...  
settlement_percentage       127561
desc                        128412
url                         128412
member_id                   128412
id                          128412
Length: 144, dtype: int64

## Missing Values
Removing the first and last two lines has cleaned up the data, however we still have several columns that are present, but include no values whatsoever, as we can see comparing their missing value count to the shape printed above.  As was discussed during the lecture, this is most likely due to the data being witheld from the table for security/privacy concerns.

To better clean and prepare the dataset, we will remove them from the table.

In [ ]:
#Remove the columns that are present in the table, but have no values in them
df = df.drop(['desc', 'url', 'member_id', 'id'], axis=1)

KeyError: ignored

In [ ]:
#Verify the columns have been removed from the table
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,50.0,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN
1,16000,16000,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,85.7,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,8425,8425,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,50.0,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,33.3,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,30000,30000,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,14.3,1,0,138354,104590,36400,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


## Missing Values (cont)
As we can see comparing the header information from before and after we have dropped the columns that contain no values.

## Cleaning up the `int_rate` column

If we look at the datatypes used by our table below, the `int_rate` column contains numerical data in an object data type.  To perform numerical operations, we need to change the data type to a float datatype.


In [ ]:
#Check table datatypes
df.dtypes

loan_amnt                  int64
funded_amnt                int64
funded_amnt_inv          float64
term                      object
int_rate                  object
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 140, dtype: object

In [ ]:
#Looking at just the first 10 entries of the int_rate column
df['int_rate'].head(10)

0     14.47%
1     14.47%
2     27.27%
3     12.98%
4     26.31%
5     17.97%
6     11.80%
7     10.72%
8     16.14%
9     12.98%
Name: int_rate, dtype: object

## Troubleshooting data types

Trying to modify data in large chunks at a time can prove cumbersom due to the extra operations that may be required to handle that much data at once.  It is much better to take a single piece of data that needs to be transformed, figure out how to transform that data, then apply that operation to the data set as a whole.  

Looking at the data above, we can see that there is a blank space before the numbers, and the addition of the '%' that will prevent casting the data point into the data type that we want, so we will create a variable that contains data in the same manner, and then work on it. 

In [ ]:
#Select a single data point to work on, then perform operations to get the data into the wanted format
bad_data = ' 14.47%'

In [ ]:
#Transforming the data to remove the preceding white space and the '%' sign from the piece of data
bad_data.strip().strip('%')

'14.47'

In [ ]:
#The data is in the correct format now, but is the wrong datatype
type(bad_data.strip().strip('%'))

str

In [ ]:
#We need to also cast the datatype to the proper type for use.
type(float(bad_data.strip().strip('%')))

float

## Function/Methods
There are many ways we can use our solution to apply it to the entire column.  One way is we can create a function that will modify the data, and then creating a For loop to iterate through the column and modify each data point, and then place the list into the dataset as a new column.

In [ ]:
#Create the modification function
def modify_int_rate(orig_value):
  return float(orig_value.strip().strip('%'))

#Create an empty list to store modified values in
mod_values = []

#For Loop for iterating through the column and creating the list of new values
for i in df['int_rate']:
  mod_values.append(modify_int_rate(i))

#Add the list to the dataset as a new column
df['int_rate_modified'] = pd.Series(mod_values)

In [ ]:
#Check to see if new column as been added
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_modified
0,20000,20000,20000.0,36 months,14.47%,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,16000,16000,16000.0,36 months,14.47%,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
2,8425,8425,8425.0,36 months,27.27%,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27
3,10000,10000,10000.0,60 months,12.98%,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98
4,30000,30000,30000.0,60 months,26.31%,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,1,0,138354,104590,36400,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26.31


In [ ]:
#We can also use .apply() to modify the data values in place
df['int_rate'] = df['int_rate'].apply(modify_int_rate)

In [ ]:
#Verify that the modification occured
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_modified
0,20000,20000,20000.0,36 months,14.47,688.13,C,C2,bus driver,4 years,OWN,52000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,0,0,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
1,16000,16000,16000.0,36 months,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,Dec-2018,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,0,0,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47
2,8425,8425,8425.0,36 months,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,Dec-2018,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,0,0,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27
3,10000,10000,10000.0,60 months,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,Dec-2018,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,1,0,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98
4,30000,30000,30000.0,60 months,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,Dec-2018,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,1,0,138354,104590,36400,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26.31


## Using `.apply()`
We're going to use the `.apply()` method to clean up the employment_title column



In [ ]:
#Looking at the top 20 employement titles
df['emp_title'].value_counts(dropna=False).head(20)

NaN                   20947
Teacher                2090
Manager                1773
Registered Nurse        952
Driver                  924
RN                      726
Supervisor              697
Sales                   580
Project Manager         526
General Manager         523
Office Manager          521
Owner                   420
Director                402
Truck Driver            387
Operations Manager      387
Nurse                   326
Engineer                325
Sales Manager           304
manager                 301
Supervisor              270
Name: emp_title, dtype: int64

In [ ]:
#Look at how many different unique employment titles there currently are
df['emp_title'].nunique()

43892

In [ ]:
#How often does emp_title contain a null value?
df['emp_title'].isnull().sum()

20947

In [ ]:
#Create a function to clean the emp_title column
def clean_emp_title(title):
  if isinstance(title, str):
    return title.strip().title()
  else:
    return "Unknown"

In [ ]:
#Applying the function to the column, this time overwriting the original column rather than creating a new one.
df['emp_title'] = df['emp_title'].apply(clean_emp_title)

In [ ]:
#Verify that the column has been cleaned by looking at the top 20 employment titles again
df['emp_title'].value_counts(dropna=False).head(20)

Unknown               20947
Teacher                2557
Manager                2395
Registered Nurse       1418
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          805
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   425
Technician              420
Engineer                412
Name: emp_title, dtype: int64

In [ ]:
#Verify that there are no null values in the column
df['emp_title'].isnull().sum()

0

## Working with Datetimes and timestampes
Often in data sets, dates and times are given in string formats, however changing these to datetimes and timestamp formats can have a lot of utility in data analysis, rather than leaving them as strings. 

In [ ]:
#Looking at which columns have dates
[col for col in df if col.endswith('_d')]

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [ ]:
#Looking at the string format of the issue_d column
df['issue_d'].head()

0    Dec-2018
1    Dec-2018
2    Dec-2018
3    Dec-2018
4    Dec-2018
Name: issue_d, dtype: object

In [ ]:
#Using Pandas to detect and tranlate each data point to the appropriate datetime object
df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format=True)

In [ ]:
#Verify that the dates have been changed to datetime objects
df['issue_d'].head()

0   2018-12-01
1   2018-12-01
2   2018-12-01
3   2018-12-01
4   2018-12-01
Name: issue_d, dtype: datetime64[ns]

In [ ]:
#Using the .dt accessor to grab just the year of the dt object
df['issue_d'].dt.year

0         2018
1         2018
2         2018
3         2018
4         2018
          ... 
128407    2018
128408    2018
128409    2018
128410    2018
128411    2018
Name: issue_d, Length: 128412, dtype: int64

In [ ]:
#Using .dt, we will create two new columns, one for the year, and one for the month
df['issue_year'] = df['issue_d'].dt.year
df['issue_month'] = df['issue_d'].dt.month

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_modified,issue_year,issue_month
0,20000,20000,20000.0,36 months,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12
1,16000,16000,16000.0,36 months,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12
2,8425,8425,8425.0,36 months,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12
3,10000,10000,10000.0,60 months,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12
4,30000,30000,30000.0,60 months,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,138354,104590,36400,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26.31,2018,12


## Additional Data Manipulation

Below, we will conduct the following operations on the data:

1. Convert the term column from string to integer.
2. Make a column named loan_status_is_great. It should contain the integer 1 if loan_status is "Current" or "Fully Paid." Else it should contain the integer 0.
4. Make last_pymnt_d_month and last_pymnt_d_year columns.

In [ ]:
#Convert the term column from string to integer
#Looking at the term column
df['term']

0          36 months
1          36 months
2          36 months
3          60 months
4          60 months
             ...    
128407     36 months
128408     36 months
128409     36 months
128410     36 months
128411     36 months
Name: term, Length: 128412, dtype: object

In [ ]:
df['term'][0]

' 36 months'

In [ ]:
#Create a function to modify the data points
def clean_term(term):
  return int(term.strip().strip(' months'))

#Test the function
clean_term(df['term'][0])

36

In [ ]:
#Apply the function to each cell in the term column
df['term'] = df['term'].apply(clean_term)

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_modified,issue_year,issue_month
0,20000,20000,20000.0,36,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,61699,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12
1,16000,16000,16000.0,36,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,116647,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12
2,8425,8425,8425.0,36,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,768304,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12
3,10000,10000,10000.0,60,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,36200,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12
4,30000,30000,30000.0,60,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,138354,104590,36400,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26.31,2018,12


In [ ]:
#Verify that the data type was changed to int
df.dtypes


loan_amnt                  int64
funded_amnt                int64
funded_amnt_inv          float64
term                       int64
int_rate                 float64
                          ...   
settlement_percentage    float64
settlement_term          float64
int_rate_modified        float64
issue_year                 int64
issue_month                int64
Length: 143, dtype: object

## Loan Status

Below we are going to make a column named loan_status_is_great.  It will contain ints, 1 if the status is "Current" or "Fully Paid", else it will contain a 0.


In [ ]:
#Create a function to check loan status and return the proper int
def check_loan_status(loan):
  if loan in ('Current', 'Fully Paid'):
    return 1
  else: 
    return 0

#Check the function
test = ['Current', 'Fully Paid', 'Other']

for t in test:
  print(check_loan_status(t))

1
1
0


In [ ]:
#Applying the function and creating the new column
new_column = []

for loan in df['loan_status']:
  new_column.append(check_loan_status(loan))

df['loan_status_is_great'] = pd.Series(new_column)

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_modified,issue_year,issue_month,loan_status_is_great
0,20000,20000,20000.0,36,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,33157,20700,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,0
1,16000,16000,16000.0,36,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,57465,32100,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,1
2,8425,8425,8425.0,36,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,189194,45800,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12,1
3,10000,10000,10000.0,60,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,27595,13000,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12,1
4,30000,30000,30000.0,60,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,104590,36400,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26.31,2018,12,0


## Creating last payment month and year calendars

Below we will create new columns for the last payment month and year.

In [ ]:
#Evaluate the last_pymnt_d column
df['last_pymnt_d']

0         Feb-2020
1         May-2020
2         Feb-2019
3         May-2020
4         Apr-2019
            ...   
128407    Dec-2018
128408    May-2020
128409    May-2020
128410    May-2020
128411    May-2020
Name: last_pymnt_d, Length: 128412, dtype: object

In [ ]:
#Change the datatype to a datetime object
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)

df['last_pymnt_d'].head()

0   2020-02-01
1   2020-05-01
2   2019-02-01
3   2020-05-01
4   2019-04-01
Name: last_pymnt_d, dtype: datetime64[ns]

In [ ]:
df['last_pymnt_d'].dt.year.astype('Int64')


0         2020
1         2020
2         2019
3         2020
4         2019
          ... 
128407    2018
128408    2020
128409    2020
128410    2020
128411    2020
Name: last_pymnt_d, Length: 128412, dtype: Int64

In [ ]:
#Create new columns and add them to the table
df['last_pymnt_year'] = df['last_pymnt_d'].dt.year.astype('Int64')
df['last_pymnt_month'] = df['last_pymnt_d'].dt.month.astype('Int64')

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_modified,issue_year,issue_month,loan_status_is_great,last_pymnt_year,last_mymnt_month,last_pymnt_month
0,20000,20000,20000.0,36,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,0,2020,2.0,2
1,16000,16000,16000.0,36,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,1,2020,5.0,5
2,8425,8425,8425.0,36,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12,1,2019,2.0,2
3,10000,10000,10000.0,60,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12,1,2020,5.0,5
4,30000,30000,30000.0,60,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26.31,2018,12,0,2019,4.0,4


In [ ]:
#Remove old bad column
del df['last_mymnt_month']

In [ ]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,int_rate_modified,issue_year,issue_month,loan_status_is_great,last_pymnt_year,last_pymnt_month
0,20000,20000,20000.0,36,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1,Jun-1979,2,15.0,NaN,6,0,15048,73%,22,w,0.00,0.00,8169.130000,8169.13,5731.54,2437.59,0.0,0.0,...,40999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688.13,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,688.13,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,0,2020,2
1,16000,16000,16000.0,36,14.47,550.51,C,C2,Carpenter,10+ years,RENT,150000.0,Source Verified,2018-12-01,Current,n,other,Other,917xx,CA,17.94,0,Nov-2001,1,NaN,NaN,24,0,43721,59.2%,29,w,9726.53,9726.53,8788.870000,8788.87,6273.47,2515.40,0.0,0.0,...,42847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,14.47,2018,12,1,2020,5
2,8425,8425,8425.0,36,27.27,345.18,E,E5,Senior Director Risk Management,3 years,MORTGAGE,450000.0,Verified,2018-12-01,Fully Paid,n,credit_card,Credit card refinancing,020xx,MA,12.37,0,Oct-1997,0,NaN,NaN,21,0,36812,65.7%,37,w,0.00,0.00,8735.148975,8735.15,8425.00,310.15,0.0,0.0,...,189054,141905.0,Jun-2013,0.0,1.0,4.0,77.4,1.0,4.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,27.27,2018,12,1,2019,2
3,10000,10000,10000.0,60,12.98,227.43,B,B5,Printer,9 years,RENT,60000.0,Not Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,660xx,KS,14.90,0,May-2007,0,NaN,112.0,7,1,10677,54.2%,12,w,7787.21,7787.21,3805.020000,3805.02,2212.79,1592.23,0.0,0.0,...,16500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,12.98,2018,12,1,2020,5
4,30000,30000,30000.0,60,26.31,903.73,E,E4,Casino Host,10+ years,OWN,62000.0,Source Verified,2018-12-01,Charged Off,n,debt_consolidation,Debt consolidation,920xx,CA,37.24,0,Oct-2007,0,NaN,86.0,15,1,18890,39.8%,36,w,0.00,0.00,2645.410000,2645.41,754.24,1891.17,0.0,0.0,...,90854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,26.31,2018,12,0,2019,4


# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [ ]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [ ]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [ ]:
# %cd instacart_2017_05_01